In [118]:
##### CONFIG

import os


# Carpetas que contienen los audios
AUDIO_FOLDER = "audio_database"
AUDIO_DATABASE = "audios3m_test"

# Carpeta de salida
RESULT_FOLDER = "resultadosBaseDatos"
RESULT_DATABASE = "pruebas_andres_25_oct.csv"


# Windows params in ms
WIN_LENGTH = 25     # windows length
WIN_OVERLAP = 10    # 10 ms of overlapping means it moves (25-10)ms = 15 ms with each step

#https://www.kaggle.com/code/ilyamich/mfcc-implementation-and-tutorial/notebook
#https://librosa.org/doc/latest/index.html


#%% Opción de testeo
#% Cuando se generan los datos de train se debe incluir la clase en la
#% ventana
#IS_TRAINING = true;


#REDUCE_SILENCES = false;   % Si se quiere eliminar silencios (ASICA tiene demasiados)
#SILENT_RATE = 0.2;         % Ratio que queda de silencios

#% Si se quiere realizar la detección de silencios al inicio y al final se
#% pone a true. Si no, a false.
#DETECT_SILENCES = false;


#% Ajuste de las vocales al 30% de las consonantes nasales
#NV_PERCENT_AFTER_BEFORE = true;
#NV_BEFORE = 0.3;  % antes
#NV_AFTER = 0.0;   % después


##### MFCC #####
# Params for mfcc calculation
NUM_MFCC = 13
##### MFCC #####

NUM_ZEROS_WINDOWS_MAX = 5

COLUMNS_NAMES_SPEAKER = ["speaker", "device"] # define los nombres de los parámetros separados por '_', uno de ellos tiene que ser speaker


#%% Definición de descriptores
FEATURES_TO_CALCULATE = ["mfcc","delta","deltaDelta"]
WIN_COLUMN_NAME = ["window"]

#% Los descriptores que se pueden calcular:
#% "mfcc"                mfcc
#% "delta"               delta mfcc
#% "deltaDelta"          delta delta
#% "vlhr"                6 mfcc con frecuencia de corte 400,500,...,900 Hz
#% "formants"            3 formantes frecuencia f1, f2, f3 Hz
#% "formantsBW"          3 ancho de banda formantes Hz
#% "formantsDist"        3 distancia entre formantes f1-f2, f1-f3, f2-f3 Hz
#% "f0"                  3 frecuencia fundamental f0, 2xf0, 3xf0
#% "intensity"           1 intensidad en dB como Praat


# Tiers del textGrid, posiciones y significado
TEXTGRID_TIER = ['WORDS','PHONES','NLCE','RMS']
# Palabra (no necesario)
# Fonemas
# Nasalancia del fonema
# RMS del intervalo del fonema



## Ajustes previos a la ejecución
if not os.path.exists(RESULT_FOLDER):
    os.mkdir(RESULT_FOLDER)

AUDIO_PATH = os.path.join(os.getcwd(),AUDIO_FOLDER,AUDIO_DATABASE)
RESULT_PATH = os.path.join(os.getcwd(),RESULT_FOLDER,RESULT_DATABASE)
RESULT_PATH_XLSX = RESULT_PATH.replace('.csv','.xlsx')

MFCC_NAMES = [f"mfcc{x}" for x in range(0,NUM_MFCC)]
DELTA_NAMES = [f"delta{x}" for x in range(0,NUM_MFCC)]
DELTA_DELTA_NAMES = [f"deltaDelta{x}" for x in range(0,NUM_MFCC)]

In [2]:
from scipy.io import wavfile
import numpy as np
import librosa          # https://librosa.org/doc/latest/index.html
from pandas import DataFrame

import textgrid
from pathlib import Path
import pandas as pd 
import numpy as np
import os
import re

import csv

from scipy.io.wavfile import write
from scipy.io import wavfile 

from python_speech_features import mfcc     # https://python-speech-features.readthedocs.io/en/latest/
from python_speech_features import delta

import scipy.io.wavfile as wav

In [75]:
def check_wav_textgrid(audio_folder):
    ''' Comprueba que la carpeta audio_folder contiene archivos .wav y .textgrid en el mismo número.
        Devuelve dos listas con los archivos ordenados 
    '''

    audio_list = [os.path.join(audio_folder,file) for file in os.listdir(audio_folder) if file.endswith('.wav')]
    textgrid_list = [os.path.join(audio_folder,file) for file in os.listdir(audio_folder) if file.endswith('.TextGrid')]

    if not audio_list: # check if empty
        print(f"No hay archivos .wav en la carpeta: {audio_folder}")
        print("Introduce lo archivos .wav y vuelve a ejecutar el scipt.")
        exit(1)
    elif not textgrid_list: # check if empty
        print(f"No hay archivos .TextGrid en la carpeta: {audio_folder}")
        print("Introduce lo archivos .TextGrid y vuelve a ejecutar el scipt.")
        exit(1)
    elif len(audio_list) != len(textgrid_list): 
        print(f"No hay el mismo número de archivos .wav que .TextGrid en la carpeta: {audio_folder}")
        print("Cada archivo .wav tiene que venir acompañado de su archivo .TextGrid. Introduce los archivos que falten y vuelve a ejecutar el script.")
        exit(1)
    else:
        print(f"Número de archivos .wav y .TextGrid correcto, continúa la ejecución del script.")

    return sorted(audio_list), sorted(textgrid_list)


def get_speakeraudio_file(audio_path):
    ''' Genera un dataframe con la información que se puede extraer del nombre del usuario. 
        La información tiene que estar separada por '_' y el nombre de cada parámetro viene definido en COLUMNS_NAMES_SPEAKER
    '''

    speaker_list = (os.path.split(audio_path)[-1]).split('_') # (head,tail)
    speaker_list[-1] = speaker_list[-1].replace('.wav','')
    speaker_df = pd.DataFrame([speaker_list], columns = COLUMNS_NAMES_SPEAKER) 

    return speaker_df


def moving_window_textgrid(audio, width=WIN_LENGTH, overlap=WIN_OVERLAP):
    if type(audio) != list:
        audio = audio.tolist()
    return [max(set(audio[i : i + width]), key = audio[i : i + width].count) for i in range(0, len(audio), width-overlap)]


def compute_mfcc(audio_data, samplerate):
    ''' Calcula los mfcc en función de los parámetros definidos en config
    '''
    mfcc_feat = mfcc(signal=np.array(audio_data, dtype=np.single), samplerate=samplerate, 
                winlen=WIN_LENGTH*1e-3, winstep=(WIN_LENGTH-WIN_OVERLAP)*1e-3, numcep=NUM_MFCC, 
                nfilt=26, nfft=1024, lowfreq=0, highfreq=None, preemph=0.97, ceplifter=22, appendEnergy=True, winfunc=np.hamming)
    return mfcc_feat


def compute_delta(input_data, num_delta=2):
    ''' Calcula los delta del mfcc en función de los parámetros definidos en config
    '''
    return( delta(input_data, 2) )

In [76]:
## MAIN

# def main():

# Se comprueba que la carpeta AUDIO_PATH contiene archivos .was y .textgrid
audio_list, textgrid_list = check_wav_textgrid(AUDIO_PATH)

# Se comprueba si existe la base de datos que se va a generar
print(f"Base de datos a generar: {RESULT_PATH}")

if os.path.exists(RESULT_PATH):
    print("Existe la base de datos")

    # nomBDstr=str(nombreBaseDatos)
    # print("La BD existe")
    # featuresTodosIntervalos=pd.read_csv(nombreBaseDatos)
    # listaLocutores=calculaLocutores(featuresTodosIntervalos)
    # print("Lista de locutores analizados anteriormente:")
    # for l in listaLocutores:
    #     print(l)
    # totalVentanas=len(featuresTodosIntervalos['Row'])
    # print("Número de ventanas en la BD original: ",totalVentanas)
else:
    print("La base de datos NO existe. Se crea una nueva.")

    speakers_list = []
    features_already_generated = []
    
    # listaLocutores=[]
    # featuresTodosIntervalos=[]
    # totalVentanas=0
    # feature_list = []
    #compute_audio_features()

    # https://stackoverflow.com/questions/13784192/creating-an-empty-pandas-dataframe-and-then-filling-it

# ANTES DE CALCULAR LOS ELEMENTOS HAY QUE COMPROBAR QUE FEATURES_TO_CALCULATE NO ESTÁ VACÍA


Número de archivos .wav y .TextGrid correcto, continúa la ejecución del script.
Base de datos a generar: d:\OneDrive\nasalance\resultadosBaseDatos\pruebas_andres_25_oct.csv
Existe la base de datos


In [185]:
features_calculated_forloop = [] # descriptores calculados al finalizar el bucle for


for audio_file, textgrid_file in zip(audio_list,textgrid_list):

    features_calculated_speaker = [] # descriptores calculados para este speaker

    # Datos que se generan para cada audio 
    speaker_info_df = get_speakeraudio_file(audio_file)
    spk_loop = speaker_info_df["speaker"].values[0]

    # Comprueba si el speaker se encuentra en la base de datos
    if spk_loop in speakers_list:
        print(f"Locutor {spk_loop} ya se encuentra en la base de datos. Pasamos al siguiente.")
    else:
        print(f"Locutor {spk_loop} NUEVO. Se calculan los descriptores.")

        # Se lee el audio
        samplerate, audio_data = wavfile.read(audio_file)


        ## Primero se calculan los descriptores
        # MFCC
        if 'mfcc' in FEATURES_TO_CALCULATE:
            mfcc_feat = compute_mfcc(audio_data, samplerate)
            features_calculated_speaker.append(mfcc_feat)

        # Delta
        if ('mfcc' in FEATURES_TO_CALCULATE) and ('delta' in FEATURES_TO_CALCULATE):
            delta_feat = compute_delta(mfcc_feat, num_delta=2)
            features_calculated_speaker.append(delta_feat)

        # Delta delta
        if ('mfcc' in FEATURES_TO_CALCULATE) and ('delta' in FEATURES_TO_CALCULATE) and ('deltaDelta' in FEATURES_TO_CALCULATE):
            delta_delta_feat = compute_delta(delta_feat, num_delta=2)
            features_calculated_speaker.append(delta_delta_feat)

        # Completar resto de descriptores
        # POR COMPLETAR



        ## Número de ventanas
        increasing_win_index = [f"w{str(x).zfill(NUM_ZEROS_WINDOWS_MAX)}" for x in range(0,features_calculated_speaker[0].shape[0])]
        features_calculated_speaker.insert(0, np.c_[increasing_win_index])    
            

        ## Datos del locutor
        # Los datos del locutor se introducen en sentido inverso para que aparezcan al principio
        for elem in reversed(speaker_info_df.columns): 
            features_calculated_speaker.insert(0, np.full((features_calculated_speaker[0].shape[0], 1), speaker_info_df[elem]))


        ## Textgrid
        textgrid_features = compute_textgrid(textgrid_file, samplerate, len(features_calculated_speaker[0]))
        features_calculated_speaker.append(textgrid_features)


        ## Datos de clase


        # Se concatenan por columnas [A B C D]
        # features_calculated_speaker es un array de columnas
        features_calculated_forloop.append(np.concatenate(features_calculated_speaker, axis=1)) # se concatenan por columnas [A B C D]
    #end if
# end for

# Se concatenan por filas [A 
#                          B] y se convierte en un único array
features_calculated_rows = np.concatenate(features_calculated_forloop, axis=0)

# Se convierte a dataframe 
features_df = pd.DataFrame(features_calculated_rows, 
                            columns = [list(speaker_info_df.columns) + WIN_COLUMN_NAME + MFCC_NAMES + DELTA_NAMES + DELTA_DELTA_NAMES][0] + TEXTGRID_TIER)

# Se guarda como csv
features_df.to_csv(RESULT_PATH, encoding='utf8', index=False)  

print(f"\nFIN calculo descriptores") 

Locutor nas20001 NUEVO. Se calculan los descriptores.
Calculando Textgrid.
Locutor nas20002 NUEVO. Se calculan los descriptores.
Calculando Textgrid.
Locutor nas20003 NUEVO. Se calculan los descriptores.
Calculando Textgrid.

FIN calculo descriptores


In [184]:
features_df.head()

,speaker,device,window,mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,...,deltaDelta7,deltaDelta8,deltaDelta9,deltaDelta10,deltaDelta11,deltaDelta12,WORDS,PHONES,NLCE,RMS
0,nas20001,iphone,w00000,9.166181,-23.231655,-6.049424,-4.11619,-4.747366,-3.538267,-3.089952,...,0.257634,-0.010671,1.208101,1.037282,1.19364,0.48146,,,,25
1,nas20001,iphone,w00001,8.59443,-18.461415,-7.247664,0.486281,-1.830832,6.670384,8.372136,...,0.888357,-0.006112,1.7736,0.608121,1.66106,1.47106,,,,25
2,nas20001,iphone,w00002,8.277615,-19.190404,-8.235758,0.463294,-1.16536,-4.990079,-7.993775,...,1.262788,-0.236115,2.268397,0.746853,1.844205,1.706149,,,,25
3,nas20001,iphone,w00003,7.866211,-13.83648,-5.748606,5.953777,-1.23124,-11.587506,-15.052852,...,0.16513,-0.601665,0.906591,-0.011343,1.020805,1.146006,,,,25
4,nas20001,iphone,w00004,7.982168,-13.16407,-6.478522,3.025342,6.359977,9.949424,5.697837,...,-1.92295,-1.766915,-1.608825,-0.088923,0.574471,0.338284,,,,25


In [187]:
list(set(features_df['PHONES']))

['',
 'x',
 'p',
 'b',
 'j',
 'o',
 'l',
 't̪',
 'i',
 'a',
 'tʃ',
 'k',
 'u',
 'n',
 'β',
 'ŋ',
 'ɲ',
 'ð',
 'ɾ',
 'ʝ',
 'r',
 'm',
 'e',
 'ɣ',
 's',
 'd̪',
 'θ',
 'f',
 'w',
 'spn']

In [57]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9761 entries, 0 to 9760
Data columns (total 45 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   speaker       9761 non-null   object
 1   device        9761 non-null   object
 2   mfcc0         9761 non-null   object
 3   mfcc1         9761 non-null   object
 4   mfcc2         9761 non-null   object
 5   mfcc3         9761 non-null   object
 6   mfcc4         9761 non-null   object
 7   mfcc5         9761 non-null   object
 8   mfcc6         9761 non-null   object
 9   mfcc7         9761 non-null   object
 10  mfcc8         9761 non-null   object
 11  mfcc9         9761 non-null   object
 12  mfcc10        9761 non-null   object
 13  mfcc11        9761 non-null   object
 14  mfcc12        9761 non-null   object
 15  delta0        9761 non-null   object
 16  delta1        9761 non-null   object
 17  delta2        9761 non-null   object
 18  delta3        9761 non-null   object
 19  delta4

In [126]:
features_calculated_speaker

[array([['nas20001'],
        ['nas20001'],
        ['nas20001'],
        ...,
        ['nas20001'],
        ['nas20001'],
        ['nas20001']], dtype=object),
 array([['iphone'],
        ['iphone'],
        ['iphone'],
        ...,
        ['iphone'],
        ['iphone'],
        ['iphone']], dtype=object),
 array([['w00000'],
        ['w00001'],
        ['w00002'],
        ...,
        ['w09758'],
        ['w09759'],
        ['w09760']], dtype='<U6'),
 array([[  9.16618058, -23.23165494,  -6.04942365, ...,  -8.76536719,
          14.29689517,  12.92363629],
        [  8.59442981, -18.46141468,  -7.24766387, ...,  -9.94934649,
          -5.81185816,  -4.07516825],
        [  8.27761455, -19.19040359,  -8.2357577 , ..., -17.06369262,
          -2.83458349,  -1.84896261],
        ...,
        [ 11.30995274, -14.98008319,  -1.27029878, ...,  -4.64522826,
           6.86139528,  -8.59116388],
        [  9.83066789,  -7.91525265, -10.87601195, ...,  -7.1152238 ,
          -1.63777043,  -4.

In [11]:
textgrid_global = []
textgrid_df = pd.DataFrame(textgrid_global, columns = [TEXTGRID_TIER])
textgrid_df.head()

,WORDS,PHONES,NLCE,RMS


In [146]:
textgrid_file

'd:\\OneDrive\\nasalance\\audio_database\\audios3m_test\\nas20001_iphone.TextGrid'

In [147]:
len(features_calculated_speaker[0])

9761

In [183]:
def compute_textgrid(textgrid_file, samplerate, len_audio):

    print("Calculando Textgrid.")
    tg = textgrid.TextGrid.fromFile(textgrid_file)

    win_length_samples = int(WIN_LENGTH * samplerate / 1000) # ms
    win_overlap_samples = int(WIN_OVERLAP * samplerate / 1000) # ms

    txtgrid_speaker = []

    for interval_tier in tg:
        #print(f"Calculando IntervalTier {re.search('IntervalTier (.*), ', str(interval_tier)).group(1)}")
        tier_list = []

        for interval in interval_tier: # last annotation is invalid
            tier_list += [interval.mark]*int(np.round((interval.maxTime-interval.minTime)*samplerate))
        # end for

        win_tier_list = moving_window_textgrid(tier_list, win_length_samples, win_overlap_samples)
        txtgrid_speaker.append(win_tier_list[0:len_audio])
    # end for

    # Con la misma longitud y se convierten en las listas en columnas
    txtgrid_speaker = list(map(list, zip(*txtgrid_speaker)))
    np_speaker=np.array(txtgrid_speaker)

    return np_speaker

In [172]:
textgrid_features = compute_textgrid(textgrid_file, samplerate, len(features_calculated_speaker[0]))


Calculando Textgrid 9761
Calculando IntervalTier words
9849
Calculando IntervalTier phones
9849
Calculando IntervalTier PhonesNasalance
9849
Calculando IntervalTier totalRms
9849


In [174]:
len(textgrid_features)

9761

In [161]:
len(textgrid_features)

9849

In [39]:
len_audio

9761

In [40]:
print(len(np_speaker[0]))
print(len(np_speaker[1]))
print(len(np_speaker[2]))
print(len(np_speaker[3]))

4
4
4
4


In [41]:
np_speaker

array([['', '', '', '25'],
       ['', '', '', '25'],
       ['', '', '', '25'],
       ...,
       ['', '', '', '1'],
       ['', '', '', '1'],
       ['', '', '', '1']], dtype='<U13')

In [89]:
audio = ['a']*10000


3259872.0

551
220


In [16]:
win_tier_list = moving_window_textgrid(tier_list, win_length_samples, win_overlap_samples)

In [17]:
print(f"el audio tiene {len(audio_data)} muestras")
print(f"el textgrid generado tiene {len(tier_list)} muestras")
print(f"la diferencia es {len(audio_data)-len(tier_list)} muestras")


print(f"\nel mfcc tiene {len(features_df.index)} ventanas")
print(f"el textgrid generado tiene {len(win_tier_list)} ventanas")

el audio tiene 3230870 muestras
el textgrid generado tiene 3230857 muestras
la diferencia es 13 muestras

el mfcc tiene 9761 ventanas
el textgrid generado tiene 9761 ventanas


In [14]:
# LIBROSA
f_mfcc = librosa.feature.mfcc(y=np.array(audio_data, dtype=np.single), sr=samplerate, n_mfcc=NUM_MFCC, 
                            win_length=win_length_samples, hop_length=(win_length_samples-win_overlap_samples))

print(f_mfcc.shape)

(14, 9761)


In [17]:
mfcc_feat = mfcc(signal=np.array(audio_data, dtype=np.single), samplerate=samplerate, 
                winlen=WIN_LENGTH*1e-3, winstep=(WIN_LENGTH-WIN_OVERLAP)*1e-3, numcep=NUM_MFCC, 
                nfilt=26, nfft=1024, lowfreq=0, highfreq=None, preemph=0.97, ceplifter=22, appendEnergy=True, winfunc=np.hamming)

print(mfcc_feat.shape)

(9761, 14)


In [22]:
print(f_mfcc[:,0])
print(mfcc_feat[0,:])

[ 3.7847467e+02  3.4444511e+01  2.2362719e+01  1.5456270e+01
  1.0243723e+01  8.4755859e+00  7.3981838e+00  3.4129996e+00
 -8.0048013e-01 -2.6222309e-01 -9.6232146e-01  5.0894299e+00
  8.5226669e+00  7.7294846e+00]
[  9.16618058 -23.23165494  -6.04942365  -4.11618962  -4.74736627
  -3.53826707  -3.08995233   4.67442104 -23.19542051  11.88455044
  -8.76536719  14.29689517  12.92363629   1.83140325]


In [15]:
WIN_LENGTH*1e-3

0.025

In [ ]:
#  Empiezan los cálculos para cada archivo
for file in audiosAsica:
	fileStr=str(file)
	contador=1

	ruta, archivoAudio = os.path.split(file)
	textGridStr=fileStr.replace(".wav",".TextGrid")
	rate,sig = wavfile.read(file)

	if archivoAudio.endswith(".wav"):
		archivoAudio=archivoAudio[:-4]
	print("Nuevo locutor: ", archivoAudio)
	if (archivoAudio in listaLocutores):
		print("-- Locutor repetido. Pasamos al siguiente")
		continue
	if (os.path.isfile(textGridStr)):
		grid=textgrid.TextGrid.fromFile(textGridStr)
		rutaT, archivoT = os.path.split(textGridStr)
		print("-- Leído el textGrid: ",archivoT)
	else:
		rutaT, archivoT = os.path.split(textGridStr)
		print("No se ha podido abrir el textGrid: ",textGridStr)
		print("-- Se ignora el wav. Pasamos al siguiente")
		continue


	# Bucle para calcular ventanas de cada clase
	numV_OC=0
	numV_OV=0
	numV_NC=0
	numV_NV=0
	numV_SL=0

	# grid=textgrid.TextGrid.fromFile(textGridStr)
	
	nIntervalos=len(grid[tierRMS].intervals)-1
	for intervalo in (range(nIntervalos)):
		phone=grid[tierPhones][intervalo].mark
		rms=int(grid[tierRMS][intervalo].mark)
		if len(grid[tierNlce][intervalo].mark)>0:
			nlceText=grid[tierNlce][intervalo].mark
			nlceText=nlceText.split('.')[0]
			nlce=int(nlceText)
		else:
			nlce=0
		sc=subclas(phone, nlce, rms)
		min=grid[tierRMS][intervalo].minTime
		max=grid[tierRMS][intervalo].maxTime
		duracion=max-min
		nv=int(round((duracion-0.015)/0.01))
		if sc=="SL":
			numV_SL=numV_SL+nv
		elif sc=="OV":
			numV_OV=numV_OV+nv
		elif sc=="OC":
			numV_OC=numV_OC+nv
		elif sc=="NV":
			numV_NV=numV_NV+nv
		elif sc=="NC":
			numV_NC=numV_NC+nv
		else:
			numV_NC=numV_NC

	# zz por aquí voy. Ahora tengo que controlar el número de ventanas de cada clase. y no seguir si ha llegado al límite
	maximos=[numV_NC, numV_OC, numV_NV, numV_OV,numV_SL]
	# print(maximos)
	elementoMenor=np.argmin(maximos)
	elementoMayor=np.argmax(maximos)

	if balanceado:
		maxVentanas=maximos[elementoMenor]
		if numClases==3:
			maxVentanasSil=maxVentanas*2
		elif numClases==5:
			maxVentanasSil=maxVentanas
		else:
			print("Error: solo puede hacer tres o cinco clases")
			exit()
	else:
		maxVentanas=maximos[elementoMayor]


	print("Max. NC: ", numV_NC)
	print("Max. OC: ", numV_OC)
	print("Max. NV: ", numV_NV)
	print("Max. OV: ", numV_OV)
	print("Max. SL: ", numV_SL)
	print("Max ventanas: ",maxVentanas)

	numV_NC=0
	numV_OC=0
	numV_NV=0
	numV_OV=0
	numV_SL=0

	nIntervalos=len(grid[tierRMS].intervals)-1
	for intervalo in (range(nIntervalos)):
		if (intervalo > -1):
			min=grid[tierRMS][intervalo].minTime
			max=grid[tierRMS][intervalo].maxTime
			duracion=max-min
			nv=int(round((duracion-0.015)/0.01))
			beginFragment=int(round(min*rate))
			endFragment=int(round(max*rate))
			fragmento=sig[beginFragment:endFragment]
			phone=grid[tierPhones][intervalo].mark
			rms=int(grid[tierRMS][intervalo].mark)
			if len(grid[tierNlce][intervalo].mark)>0:
				nlceText=grid[tierNlce][intervalo].mark
				nlceText=nlceText.split('.')[0]
				nlce=int(nlceText)
			else:
				nlce=0
			
			sc=subclas(phone, nlce, rms)
			
			# Si ya tenemos el máximo de ventanas de una cat, pasamos al sig. intervalo
			if sc=="SL":
				if numV_SL > maxVentanasSil:
					continue
				else:
					numV_SL=numV_SL+nv
			if sc=="OC":
				if numV_OC > maxVentanas:
					continue
				else:
					numV_OC=numV_OC+nv
			if sc=="OV":
				if numV_OV > maxVentanas:
					continue
				else:
					numV_OV=numV_OV+nv
			if sc=="NC":
				if numV_NC > maxVentanas:
					continue
				else:
					numV_NC=numV_NC+nv
			if sc=="NV":
				if numV_NV > maxVentanas:
					continue
				else:
					numV_NV=numV_NV+nv


			if duracion>0 and len(featuresTodosIntervalos)==0 and sc!="":
				features=allFeatures(rate,fragmento) # Archivo de Audio
				features=defineColumnaRow(features, sc, archivoAudio, contador)
				featuresTodosIntervalos=features
				contador=contador+len(features)
				# print("Ventana inicial: ",contador,"Sc:",sc, " Inicio: ",min," Fin: ",max)

			elif duracion>0 and len(featuresTodosIntervalos)>0 and sc!="":				
				features=allFeatures(rate,fragmento) # Archivo de Audio
				features=defineColumnaRow(features, sc, archivoAudio, contador)
				contador=contador+len(features)
				featuresTodosIntervalos = pd.concat([featuresTodosIntervalos, features])
				# print("Ventana inicial: ",contador,"Sc:",sc, " Inicio: ",min," Fin: ",max)
			else:
				a=1
				#print("Intervalo no anotado: se ignora. Inicio: ",min," Fin: ",max)
	totalVentanas=totalVentanas+contador
	ventanasNuevas=ventanasNuevas+contador
	print("-- Ventanas procesadas en el último audio: ",contador)
	print("-- Ventanas procesadas en total: ", totalVentanas)
		


In [10]:
not len(audio_list)

False

In [3]:
def moving_window(audio, width=1000, overlap=400):
    audio = audio.tolist()
    return [audio[i : i + width] for i in range(0, len(audio), width-overlap)]

In [4]:
samplerate, audio_data = wavfile.read(AUDIO_PATH)
print(len(audio_data))
# Transform length and overlap from ms to samples
win_length_samples = int(WIN_LENGTH * samplerate / 1000)
win_overlap_samples = int(WIN_OVERLAP * samplerate / 1000)
print(win_length_samples)
print(win_overlap_samples)

audio_win =  moving_window(audio_data, width=win_length_samples, overlap=win_overlap_samples)
print(type(audio_win))

665571
1102
441
<class 'list'>


In [7]:
# def compute_features(audio, samplerate):
audio = audio_win

feat_audio = []

#for chunk in audio:
chunk = audio[0]
if 'mfcc' in FEATURES:
    # Calculate mfcc for audio
    librosa.feature.mfcc(y=audio_win, sr=samplerate, n_mfcc=NUM_MFCC, dct_type=2, norm='ortho', lifter=0)

elif 'delta' in FEATURES:
    # do nothing
    8


SyntaxError: invalid character '¶' (U+00B6) (600687401.py, line 10)

In [36]:
f_mfcc = librosa.feature.mfcc(y=np.array(audio_data, dtype=np.single), sr=samplerate, n_mfcc=NUM_MFCC, 
                            win_length=win_length_samples)

print(f_mfcc)
print(len(f_mfcc[0]))                        
print(f_mfcc.shape)

[[ 4.86895996e+02  5.04622528e+02  4.94123230e+02 ...  3.78741302e+02
   3.73278259e+02  3.86671753e+02]
 [ 8.32832870e+01  8.94101410e+01  8.41661148e+01 ...  1.53775215e+01
   9.10860062e+00  2.74301033e+01]
 [-1.21022902e+01 -2.49979515e+01 -2.47444077e+01 ...  1.08155813e+01
   7.34319687e+00  2.38158760e+01]
 ...
 [-5.59439242e-01 -7.57081389e-01  2.49762490e-01 ... -1.51912427e+00
  -1.48723435e+00  1.65443528e+00]
 [-9.99189377e+00 -7.82266235e+00 -5.94357204e+00 ... -1.91025078e+00
  -1.23540735e+00  9.63437200e-01]
 [-1.47784491e+01 -1.05038767e+01 -9.53153038e+00 ... -2.38252163e+00
  -1.03475046e+00  8.92013788e-01]]
1300
(14, 1300)


In [29]:
len(np.array(audio_data, dtype=np.single))

665571